In [35]:
import sys
sys.path.append('../src')

import pandas as pd
import numpy as np

from utils.data_utils.data_handler import Handler
from backtest.backtest import Backtest

import time

from glob import glob
import shutil
import logging

In [36]:
benchmark_csv_dir = '../data/benchmark/dc_performance.csv'
fundamental_csv_dir = '../data/fundamental_data/'
historical_csv_dir = '../data/historical_data/'
gics_csv_dir = '../data/gics/gics.csv'

handl = Handler(benchmark_csv_dir, fundamental_csv_dir, historical_csv_dir, gics_csv_dir)

In [37]:
benchmark = handl.get_benchmark_data() 
fun_data = handl.get_processed_fundamental_data() 
hist_data = handl.get_processed_historical_data() 

In [38]:
def get_transform_data(benchmark, fun_data, hist_data, start_time='2013-12-31'): 
    idx = pd.IndexSlice
    daily = hist_data.copy()
    
    long_daily = daily.reset_index().set_index(['time', 'ticker'])
    long_daily['r'] = long_daily.groupby(level=1)['close'].transform(lambda x: x.pct_change())

    monthly_close = long_daily.unstack()['close'].resample('ME').last().stack(future_stack=True).to_frame(name='close')
    monthly_vol = long_daily.unstack()['volume'].resample('ME').sum().stack(future_stack=True).to_frame(name='volume')
    monthly_ret = monthly_close.unstack()['close'].pct_change().stack(future_stack=True).to_frame(name='r')

    long_monthly = pd.merge(monthly_vol, monthly_ret, how='inner', left_index=True, right_index=True)
    long_monthly = pd.merge(long_monthly, monthly_close, how='inner', left_index=True, right_index=True)
    long_monthly['yearReport'] = long_monthly.index.get_level_values(0).year
    long_monthly['lengthReport'] = long_monthly.index.get_level_values(0).quarter

    long_monthly = long_monthly.loc[idx[start_time:, :], :]
    long_monthly = long_monthly.reset_index()

    long_monthly = pd.merge(long_monthly, fun_data, how='left', on=['ticker', 'yearReport', 'lengthReport'])
    long_monthly = pd.merge(long_monthly, benchmark, how='left', on=['time'])
    long_monthly['book_to_market'] = 1 / long_monthly['P/B']
    long_monthly['log_mcap'] = np.log(long_monthly['close']*long_monthly['Outstanding Share'])

    long_monthly = long_monthly.set_index(['time', 'ticker'])
    long_daily = long_daily.sort_index(level=0)
    
    return long_daily, long_monthly

In [39]:
long_daily, long_monthly = get_transform_data(benchmark, fun_data, hist_data, start_time='2014-01-01')

/tmp/ipykernel_169456/2955078845.py:6: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  long_daily['r'] = long_daily.groupby(level=1)['close'].transform(lambda x: x.pct_change())
/tmp/ipykernel_169456/2955078845.py:10: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  monthly_ret = monthly_close.unstack()['close'].pct_change().stack(future_stack=True).to_frame(name='r')
/root/quant-trading/principle-component-mommentum/.venv/lib/python3.12/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [40]:
bt = Backtest(long_daily, long_monthly)

In [41]:
past_formation_dfs, future_holding_dfs = bt.get_rolling_df()

In [ ]:
def filter_condition_1(df): 
    idx = pd.IndexSlice
    tmp = df.copy()
    selected_stocks = set(tmp['book_to_market'].dropna(axis=1).columns)
    
    return selected_stocks

def filter_condition_2(df): 
    idx = pd.IndexSlice
    tmp = df.copy()
    
    # 1. All return data to calculate the momentum measures in the subperiod are available 
    selected_stocks = set(tmp['r'].dropna(axis=1).columns)

    # 2. All price data and the number of shares ourstanding in the past period are available to calculate investment weight and market capitalization 
    selected_stocks = selected_stocks.intersection(tmp['log_mcap'].dropna(axis=1).columns)

    # 3. Among stocks with all trading volume data during the past formation period, stocks with valid non-zero return data on 50% or more trading days are available 
    check = long_daily.loc[idx[tmp.index.min()-pd.offsets.MonthEnd()+pd.DateOffset(1):tmp.index.max(), :], :].unstack()
    na_less_than_5_pct = (check['volume'].isnull().sum() / len(check) < .05)
    satis_cond = na_less_than_5_pct[na_less_than_5_pct == True].index
    selected_stocks = selected_stocks.intersection(check['volume'][satis_cond].dropna(axis=0).columns)

    valid_df = (check['r'][list(selected_stocks)].eq(0).sum() / len(check['r'][list(selected_stocks)]) < .5).to_frame(name='valid_r')
    selected_stocks = set(valid_df[valid_df['valid_r'] == True].index)
    
    return selected_stocks

def filter_condition_3(df, stocks):
    idx = pd.IndexSlice
    tmp = df.copy()
    stocks = list(stocks)

    check = tmp.loc[:, idx[:, stocks]]
    daily_subset = long_daily.loc[idx[tmp.index.min()-pd.offsets.MonthEnd()+pd.DateOffset(1):tmp.index.max(), :], :].unstack()['r'][stocks]
    
    volatility = daily_subset.replace(np.inf, np.nan).std()
    mcap = check['log_mcap'].iloc[-1]

    # 1. Exclude stocks with extreme volatility
    volatility_95th_percentile = volatility.quantile(.95)
    valid_volatility = (volatility <= volatility_95th_percentile).to_frame(name='valid_vol')
    filtered_volatility = set(valid_volatility[valid_volatility['valid_vol'] == True].index)

    # 2. Exclude micro-small firms
    mcap_5th_percentile = mcap.quantile(.05)
    valid_mcap = (mcap >= mcap_5th_percentile).to_frame(name='valid_cap')
    filtered_mcap = set(valid_mcap[valid_mcap['valid_cap'] == True].index)
    
    selected_stocks = filtered_volatility.intersection(filtered_mcap)
    
    return selected_stocks


def get_stock_selected_in_subperiod(dfs):
    subperiod_selected_stocks = []
    
    pass_cond2_tmp = []
    
    for i, period in enumerate(dfs): 
        print(f"Period: {i}")
        # Filter condition 1
        passed_cond_1 = filter_condition_1(period)

        # Filter condition 2
        passed_cond_2 = passed_cond_1.intersection(filter_condition_2(period))

        # Filter condition 3
        passed_cond_3 = filter_condition_3(period, passed_cond_2) # => Runtime warning come from condition 3
        
        subperiod_selected_stocks.append(list(passed_cond_3))

    return subperiod_selected_stocks
        
subperiod_selected_stocks = get_stock_selected_in_subperiod(past_formation_dfs)

Period: 0
Period: 1
Period: 2
Period: 3
Period: 4
Period: 5
Period: 6
Period: 7
Period: 8
Period: 9
Period: 10
Period: 11
Period: 12
Period: 13
Period: 14
Period: 15
Period: 16
Period: 17
Period: 18
Period: 19
Period: 20
Period: 21
Period: 22
Period: 23
Period: 24
Period: 25
Period: 26
Period: 27
Period: 28
Period: 29
Period: 30
Period: 31
Period: 32
Period: 33
Period: 34
Period: 35
Period: 36
Period: 37
Period: 38
Period: 39
Period: 40
Period: 41
Period: 42
Period: 43
Period: 44
Period: 45
Period: 46
Period: 47
Period: 48
Period: 49
Period: 50
Period: 51
Period: 52
Period: 53
Period: 54
Period: 55
Period: 56
Period: 57
Period: 58
Period: 59
Period: 60
Period: 61
Period: 62
Period: 63
Period: 64
Period: 65
Period: 66
Period: 67
Period: 68
Period: 69
Period: 70
Period: 71
Period: 72
Period: 73
Period: 74
Period: 75
Period: 76
Period: 77
Period: 78
Period: 79
Period: 80
Period: 81
Period: 82
Period: 83
Period: 84
Period: 85
Period: 86
Period: 87
Period: 88
Period: 89
Period: 90
Period: 9

In [65]:
len(subperiod_selected_stocks[23])

450

In [95]:
idx = pd.IndexSlice

stocks = subperiod_selected_stocks[23]
tmp =  past_formation_dfs[23]

check = tmp.loc[:, idx[:, stocks]]
daily_subset = long_daily.loc[idx[tmp.index.min()-pd.offsets.MonthEnd()+pd.DateOffset(1):tmp.index.max(), :], :].unstack()['r'][stocks]

volatility = daily_subset.std()

/root/quant-trading/principle-component-mommentum/.venv/lib/python3.12/site-packages/pandas/core/nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)
